In [2]:
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
from sklearn.grid_search import ParameterGrid
import xgboost as xgb
import pickle
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

Read in just the testing and training data.

In [3]:
error = pd.read_pickle('scripts/cv_results/actions_e20/search3/res1.p')
error2 = pd.read_pickle('scripts/cv_results/actions_e20/search3/res2.p')
error3 = pd.read_pickle('scripts/cv_results/actions_e20/search3/res3.p')
param = pd.read_pickle('scripts/cv_results/actions_e20/params_search3.p')
#error['mean'] = error.apply(np.mean, axis = 1)
# error = error.astype(np.float64)
# error['mean'] = error.mean(1)
# error['std'] = error.std(1)

In [12]:
e = pd.read_pickle('scripts/cv_results/actions_e20/errors_search3.p')

In [61]:
param['details'] = ['actions2', 'actions2', 'actions3', 'actions3_dropped_p4total']

In [64]:
param.to_pickle('scripts/cv_results/actions_e20/params_search3.p')

In [4]:
param

,subsample,eta,colsample_bytree,max_depth,details
0,0.7,0.16,0.3,6,actions2
1,0.7,0.16,0.6,6,actions2
2,0.7,0.16,0.6,6,actions3
3,0.7,0.16,0.6,6,actions3_dropped_p4total


In [9]:
df_params.append(p, ignore_index = True)

,subsample,eta,colsample_bytree,max_depth,details
0,0.7,0.16,0.3,6,actions2
1,0.7,0.16,0.6,6,actions2
2,0.7,0.16,0.6,6,actions3
3,0.7,0.16,0.6,6,actions3_dropped_p4total
4,0.9,0.13,0.6,6,NaN


In [6]:
param_grid = {}
param_grid['eta'] = [.16]
param_grid['max_depth'] = [6]
param_grid['subsample'] = [.85, 1]
param_grid['colsample_bytree'] = [.4, .6]
nrounds = 100
 
df_params = pd.read_pickle('scripts/cv_results/actions_e20/params_search3.p')

for cnt, p in enumerate(list(ParameterGrid(param_grid)), 4):
    print p

{'subsample': 0.9, 'eta': 0.13, 'colsample_bytree': 0.4, 'max_depth': 6}
{'subsample': 0.9, 'eta': 0.13, 'colsample_bytree': 0.5, 'max_depth': 6}
{'subsample': 0.9, 'eta': 0.13, 'colsample_bytree': 0.6, 'max_depth': 6}


In [22]:
error['mean'] = error.astype(np.float).mean(axis = 1)

In [26]:
e.loc[50:,e.columns.str.contains('valid.*mean')]

,valid-error-mean0,valid-error-mean1,valid-error-mean2,valid-error-mean3
50,0.141415,0.140400,0.140647,0.140375
51,0.141493,0.140308,0.140737,0.140373
52,0.141318,0.140390,0.140692,0.140368
53,0.141280,0.140350,0.140703,0.140422
54,0.141170,0.140312,0.140748,0.140342
55,0.140960,0.140433,0.140738,0.140282
56,0.140995,0.140422,0.140677,0.140335
57,0.141025,0.140513,0.140637,0.140393
58,0.141018,0.140500,0.140587,0.140315
59,0.141055,0.140495,0.140718,0.140420


In [11]:
param_grid = {}
param_grid['eta'] = [.16]
param_grid['max_depth'] = [6]
param_grid['subsample'] = [.7, .9]
param_grid['colsample_bytree'] = [.6]
nrounds = 80
for cnt, p in enumerate(list(ParameterGrid(param_grid)), 2):
    print p

{'subsample': 0.7, 'eta': 0.16, 'colsample_bytree': 0.6, 'max_depth': 6}
{'subsample': 0.9, 'eta': 0.16, 'colsample_bytree': 0.6, 'max_depth': 6}


In [3]:
#train_100 = pd.read_csv('Airbnb_data/train_users_2.csv', nrows = 100, index_col='id')
train_data = pd.read_csv('data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('Airbnb_data/age_gender_bkts.csv')
#countries = pd.read_csv('Airbnb_data/countries.csv')
all_df = pd.read_pickle('data/all_df.p')

In [163]:
for i in np.unique(test_data.first_browser):
    if i not in np.unique(train_data.first_browser):
        print i

IBrowse
Nintendo Browser
UC Browser


In [5]:
error = pd.read_pickle('scripts/cv_results.p')
params = pd.read_pickle('scripts/cv_params.p')

In [6]:
res = pd.concat([error, params], axis = 1)

In [7]:
res.sort_values(by = 'test-error-mean', axis = 0);

In [3]:
all_df = feakaggle_model(all_df)

#one hot encoding of features
print 'number of columns before one hot encoding', all_df.shape[1]
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
all_df = pd.get_dummies(all_df, columns = ohe_feats)
print 'number of columns after one hot encoding', all_df.shape[1]

number of columns before one hot encoding 17
number of columns after one hot encoding 161


In [6]:
with open('xgbmodels/sessions_e20_25n.p', 'rb') as f:
    bst = pickle.load(f)
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]

NameError: name 'all_df' is not defined

In [ ]:
xgb.to_graphviz(bst, num_trees=11)

In [ ]:
submission = pd.DataFrame(np.column_stack([idx, cntr]), columns = ['id', 'country'])
submission.to_csv('submissions/datebin_model.csv', index=False)